# Assignment 3

Import libraries and define common helper functions

In [7]:
import os
import sys
import gzip
import json
from pathlib import Path
import csv

import pandas as pd
# import s3fs
import pyarrow as pa

import pyarrow.parquet as pq

import fastavro
import pygeohash
import snappy
import jsonschema
from jsonschema.exceptions import ValidationError

Load the records from https://storage.budsc.midwest-datascience.com/data/processed/openflights/routes.jsonl.gz 

In [8]:

endpoint_url = 'https://storage.budsc.midwest-datascience.com'

current_dir = Path(os.getcwd()).absolute()
schema_dir = current_dir.joinpath('schemas')
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)


def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]

    return records

In [9]:

current_dir = Path(os.getcwd()).absolute()
schema_dir = current_dir.joinpath('schemas')
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)

src_data_dir = current_dir.parent.parent.parent.joinpath('data\processed\openflights\\routes.jsonl.gz')

print(current_dir)
print(schema_dir)
print(src_data_dir)

c:\Users\saman\git_repos\dsc650\dsc650\assignments\assignment03
c:\Users\saman\git_repos\dsc650\dsc650\assignments\assignment03\schemas
c:\Users\saman\git_repos\dsc650\data\processed\openflights\routes.jsonl.gz


In [10]:
def read_jsonl_data():
    with gzip.open(src_data_dir, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]

    return records

In [16]:
# Open the compressed zip file
with open(src_data_dir, 'rb') as fread:
	# Now open the file to write to
	json_out = results_dir.joinpath('routes.jsonl')
	with open(json_out, 'wb') as fwrite:
		fwrite.write(gzip.decompress(fread.read()))

In [11]:
myrecords = read_jsonl_data()
# print(type(myrecords))
df_list = pd.DataFrame(myrecords)
df_list.head()


,airline,src_airport,dst_airport,codeshare,equipment
0,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2965, 'name': 'Sochi Internatio...","{'airport_id': 2990, 'name': 'Kazan Internatio...",False,[CR2]
1,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966, 'name': 'Astrakhan Airpor...","{'airport_id': 2990, 'name': 'Kazan Internatio...",False,[CR2]
2,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966, 'name': 'Astrakhan Airpor...","{'airport_id': 2962, 'name': 'Mineralnyye Vody...",False,[CR2]
3,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968, 'name': 'Chelyabinsk Bala...","{'airport_id': 2990, 'name': 'Kazan Internatio...",False,[CR2]
4,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968, 'name': 'Chelyabinsk Bala...","{'airport_id': 4078, 'name': 'Tolmachevo Airpo...",False,[CR2]


In [13]:
df_list['airline'][0]

{'airline_id': 410,
 'name': 'Aerocondor',
 'alias': 'ANA All Nippon Airways',
 'iata': '2B',
 'icao': 'ARD',
 'callsign': 'AEROCONDOR',
 'country': 'Portugal',
 'active': True}

In [16]:
df_airline = pd.json_normalize(df_list['airline'])
df_src_airport = pd.json_normalize(df_list['src_airport'])
df_dst_airport = pd.json_normalize(df_list['dst_airport'])
df_final = pd.concat([df_airline,df_src_airport,df_dst_airport,df_list['codeshare'],df_list['equipment']], axis=1)
df_final.head()

,airline_id,name,alias,iata,icao,callsign,country,active,airport_id,name,...,latitude,longitude,altitude,timezone,dst,tz_id,type,source,codeshare,equipment
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,55.606201,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2]
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,55.606201,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2]
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,44.225101,43.081902,1054.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2]
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,55.606201,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2]
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,55.012600,82.650703,365.0,7.0,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2]


In [14]:
records = read_jsonl_data()
records_file = results_dir.joinpath('records_txt.txt')
with open(records_file, 'w') as f:
	json.dump(records,f)

## 3.1

### 3.1.a JSON Schema

In [8]:
def validate_jsonl_data(records):
    schema_path = schema_dir.joinpath('routes-schema.json')
    with open(schema_path) as f:
        schema = json.load(f)
    validation_csv_path = results_dir.joinpath('validate_json.csv')    
    with open(validation_csv_path, 'w') as f:    
        for i, record in enumerate(records):
            try:
                ## TODO: Validate record 
                jsonschema.validate(instance=record, schema=schema)
            except ValidationError as e:
                ## Print message if invalid record
                pass
            
validate_jsonl_data(records)

In [46]:
def validate_jsonl_data(jsondata):
	"""This function loads the given schema available"""
	schema_path = schema_dir.joinpath('routes-schema.json')
	fname = Path(schema_path).name
	# print(fname)
	# print(schema_path)
	# return fname
	with open(fname, 'r') as fsch:
		schema = json.load(fsch)
	validation_csv_path = results_dir.joinpath('validate_json.csv')
	with open(validation_csv_path, 'w') as fwrite:
		for i, record in enumerate(jsondata):
			try:
				jsonschema.validate(instance=record, schema=schema)
			except jsonschema.exceptions.ValidationError as err:
				err = f'Error in {i}-record'
				fwrite.write(err)

In [47]:
validate_jsonl_data('jj')

routes-schema.json
c:\Users\saman\git_repos\dsc650\dsc650\assignments\assignment03\schemas\routes-schema.json


'routes-schema.json'

In [42]:
# samplejson_path = results_dir.joinpath('')
# samplejson = Path(samplejson_path).name
validate_jsonl_data('sample.json')

FileNotFoundError: [Errno 2] No such file or directory: 'routes-schema.json'

### 3.1.b Avro

In [36]:
from fastavro import writer, reader, parse_schema

def create_avro_dataset(records):
    schema_path = schema_dir.joinpath('routes.avsc')
    data_path = results_dir.joinpath('routes.avro')


    ## TODO: Use fastavro to create Avro dataset
    with open(schema_path) as f:
        schema = json.load(f)
        parsed_schema = parse_schema(schema)

    # writing
    with open(data_path, 'w') as f:    
        for i, record in enumerate(records):
            try:
                ## TODO: Validate record 
                # writer(f, parsed_schema,record)
                print(i,record)
            except ValidationError as e:
                ## Print message if invalid record
                pass
        
create_avro_dataset(myrecords)

NameError: name 'myrecords' is not defined

In [35]:
from fastavro import writer, reader, parse_schema

def create_avro_dataset(records):
    schema_path = schema_dir.joinpath('routes.avsc')
    data_path = results_dir.joinpath('routes.avro')


    ## TODO: Use fastavro to create Avro dataset
    with open(schema_path) as f:
        schema = json.load(f)
        parsed_schema = parse_schema(schema)

    # writing
    with open(data_path, 'wb') as f:
            try:
                ## TODO: Validate record 
                writer(f, parsed_schema,records)
                # print(i,record)
            except ValidationError as e:
                ## Print message if invalid record
                pass
        
create_avro_dataset(records)

NameError: name 'records' is not defined

### 3.1.c Parquet

In [ ]:
def create_parquet_dataset():
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    parquet_output_path = results_dir.joinpath('routes.parquet')
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            pass
            ## TODO: Use Apache Arrow to create Parquet table and save the dataset

create_parquet_dataset()

In [23]:
input_jsonl = current_dir.joinpath('sample.json')
routes_jsonl = results_dir.joinpath('routes.jsonl')

df = pd.read_json(routes_jsonl,lines=True, orient='columns')
df.head()

,airline,src_airport,dst_airport,codeshare,equipment
0,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2965, 'name': 'Sochi Internatio...","{'airport_id': 2990, 'name': 'Kazan Internatio...",False,[CR2]
1,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966, 'name': 'Astrakhan Airpor...","{'airport_id': 2990, 'name': 'Kazan Internatio...",False,[CR2]
2,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966, 'name': 'Astrakhan Airpor...","{'airport_id': 2962, 'name': 'Mineralnyye Vody...",False,[CR2]
3,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968, 'name': 'Chelyabinsk Bala...","{'airport_id': 2990, 'name': 'Kazan Internatio...",False,[CR2]
4,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968, 'name': 'Chelyabinsk Bala...","{'airport_id': 4078, 'name': 'Tolmachevo Airpo...",False,[CR2]


In [34]:

parquet_output_path = results_dir.joinpath('routes.parquet')
input_jsonl = current_dir.joinpath('sample.json')
routes_jsonl = results_dir.joinpath('routes.jsonl')

df = pd.read_json(routes_jsonl,lines=True, orient='columns')
df.head()
bn = pd.DataFrame(df.airline.values.tolist())['airline_id']
bn
# table = pd.DataFrame(records)
# table = pa.Table.from_pandas(table)
# pq.write_table(table, 'file.parquet')
# pd.json_normalize(bn).head()

# table = pa.Table.from_pandas(df)
# pq.write_table(table, 'routes.parquet')

0          410
1          410
2          410
3          410
4          410
         ...  
67658     4178
67659    19016
67660    19016
67661    19016
67662    19016
Name: airline_id, Length: 67663, dtype: int64

In [11]:
routes_jsonl_path = results_dir.joinpath('routes.jsonl')
print(routes_jsonl_path)
file_name = Path(routes_jsonl_path).name
print(file_name)
with open(routes_jsonl_path) as datafile:
	for line in datafile.readline:
		data = json.loads(line)
	ww = pd.DataFrame(data)

ww.head()

c:\Users\saman\git_repos\dsc650\dsc650\assignments\assignment03\results\routes.jsonl
routes.jsonl


TypeError: 'builtin_function_or_method' object is not iterable

### 3.1.d Protocol Buffers

In [ ]:
sys.path.insert(0, os.path.abspath('routes_pb2'))

import routes_pb2

def _airport_to_proto_obj(airport):
    obj = routes_pb2.Airport()
    if airport is None:
        return None
    if airport.get('airport_id') is None:
        return None

    obj.airport_id = airport.get('airport_id')
    if airport.get('name'):
        obj.name = airport.get('name')
    if airport.get('city'):
        obj.city = airport.get('city')
    if airport.get('iata'):
        obj.iata = airport.get('iata')
    if airport.get('icao'):
        obj.icao = airport.get('icao')
    if airport.get('altitude'):
        obj.altitude = airport.get('altitude')
    if airport.get('timezone'):
        obj.timezone = airport.get('timezone')
    if airport.get('dst'):
        obj.dst = airport.get('dst')
    if airport.get('tz_id'):
        obj.tz_id = airport.get('tz_id')
    if airport.get('type'):
        obj.type = airport.get('type')
    if airport.get('source'):
        obj.source = airport.get('source')

    obj.latitude = airport.get('latitude')
    obj.longitude = airport.get('longitude')

    return obj


def _airline_to_proto_obj(airline):
    obj = routes_pb2.Airline()
    ## TODO: Create an Airline obj using Protocol Buffers API
    return obj


def create_protobuf_dataset(records):
    routes = routes_pb2.Routes()
    for record in records:
        route = routes_pb2.Route()
        ## TODO: Implement the code to create the Protocol Buffers Dataset

        routes.route.append(route)

    data_path = results_dir.joinpath('routes.pb')

    with open(data_path, 'wb') as f:
        f.write(routes.SerializeToString())
        
    compressed_path = results_dir.joinpath('routes.pb.snappy')
    
    with open(compressed_path, 'wb') as f:
        f.write(snappy.compress(routes.SerializeToString()))
        
create_protobuf_dataset(records)

## 3.2

### 3.2.a Simple Geohash Index

In [ ]:
def create_hash_dirs(records):
    geoindex_dir = results_dir.joinpath('geoindex')
    geoindex_dir.mkdir(exist_ok=True, parents=True)
    hashes = []
    ## TODO: Create hash index
            
create_hash_dirs(records)

In [56]:
# Assignment 3.2.a
def create_hash_dirs(records):
    geoindex_dir = results_dir.joinpath('geoindex')
    geoindex_dir.mkdir(exist_ok=True, parents=True)
    hashes = []
    for record in records:
        src_airport = record.get('src_airport', {})
        if src_airport:
            latitude = src_airport.get('latitude')
            longitude = src_airport.get('longitude')
            if latitude and longitude:
                ## TODO: use pygeohash.encode() to assign geohashes to the records and complete the hashes list
    hashes.sort()
    three_letter = sorted(list(set([entry[:3] for entry in hashes])))
    hash_index = {value: [] for value in three_letter}
    for record in records:
        geohash = record.get('geohash')
        if geohash:
            hash_index[geohash[:3]].append(record)
    for key, values in hash_index.items():
        output_dir = geoindex_dir.joinpath(str(key[:1])).joinpath(str(key[:2]))
        output_dir.mkdir(exist_ok=True, parents=True)
        output_path = output_dir.joinpath('{}.jsonl.gz'.format(key))
        with gzip.open(output_path, 'w') as f:
            json_output = '\n'.join([json.dumps(value) for value in values])
            f.write(json_output.encode('utf-8'))

IndentationError: expected an indented block (<ipython-input-56-0a5d8fad30da>, line 13)

### 3.2.b Simple Search Feature

In [ ]:
def airport_search(latitude, longitude):
    ## TODO: Create simple search to return nearest airport
    pass
    
airport_search(41.1499988, -95.91779)